In [30]:
import warnings

from tqdm import tqdm
import pandas as pd
import numpy as np
import weighted

from jre_utils.datapath import DATA_DIRECTORY_PATH
from jp_prefecture.address import JpAddressParser

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [31]:
"""
Goal: 
For the LPA data, 
index by year, prefecture and municipality, then
get the weighted average price per unit area
DONE!!
"""

lpa_data_path = f"{DATA_DIRECTORY_PATH}/core_scraped/lpa"
lpa_data_paths = {
    year: f"{lpa_data_path}/{year}.csv"
    for year in range(1971, 2024)
}
print(lpa_data_paths[2022])

lpa_pre_derived_data_path = f"{DATA_DIRECTORY_PATH}/pre_derived_lpa"
lpa_pre_derived_data_paths = {
    year: f"{lpa_pre_derived_data_path}/yearly/{year}.csv"
    for year in range(1971, 2024)
}
print(lpa_pre_derived_data_paths[2022])


'../../data/core_scraped/lpa/2022.csv'

In [32]:
# convert address to city code
addr_column = "所在及び地番"
price_per_unit_area_column = "価格(円/m²)"
area_column = "地積(m²)"

required_columns = ["year", "area_code", "area", "unit_price", "traded_area"]

In [33]:
import json
from pprint import pprint

sub_city_to_city_path = f"{DATA_DIRECTORY_PATH}/core_scraped/sub_city_to_city.json"
with open(sub_city_to_city_path) as fd:
     sub_city_to_city = json.load(fd)
     pprint(f"E.g. Maps 1101 to {sub_city_to_city["1101"]}")

area_code_to_area_path = f"{DATA_DIRECTORY_PATH}/core_scraped/area_code_to_area.json"
with open(area_code_to_area_path) as fd:
     area_code_to_area = json.load(fd)
     pprint(f"E.g. Maps 1100 to {area_code_to_area["1100"]}") 

def get_area_from_area_code(area_code):
     return area_code_to_area.get(area_code, "na" )

'E.g. Maps 1101 to 1100'
'E.g. Maps 1100 to Hokkaido Sapporo-shi'


In [34]:
def prepare_area_column(area):
    try:
        return int(area.split("(")[0].replace(",", ""))
    except:
        print("error")
        return np.NaN

def get_city_code(parser, addr):
    try:
        city_code = str(parser.parse_address(addr).cityCode)
        return sub_city_to_city.get(city_code, city_code)
    except:
        return addr

def get_area_from_area_code(area_code):
     return area_code_to_area.get(area_code, "na" )

parser = JpAddressParser(enable_town=True)

In [35]:
def custom_aggregate(x):
    d = {}
    d["unit_price_wmean"] = np.average(x["unit_price"], weights=x["trade_area"])
    d["unit_price_wmedian"] = weighted.median(x["unit_price"], weights=x["trade_area"])
    d["unit_price_mean"] = x["unit_price"].mean()
    d["unit_price_median"] = x["unit_price"].median()
    d["total_traded_area"] = x["trade_area"].sum()
    d["count"] = x["count"].count()
    return pd.Series(
        d,
        index=[
            "unit_price_wmean",
            "unit_price_wmedian",
            "unit_price_mean",
            "unit_price_median",
            "total_traded_area",
            "count",
        ],
    )

In [37]:
%%capture

for year, data_path in lpa_data_paths.items():
    main_df = pd.read_csv(data_path)
    main_df = main_df.assign(year=year, count=1)
    main_df["area_code"] = main_df[addr_column].apply(lambda x: get_city_code(parser, x))
    main_df["area"] = main_df["area_code"].apply(get_area_from_area_code)
    main_df["unit_price"] = main_df[price_per_unit_area_column].apply(prepare_area_column)
    main_df["traded_area"] = main_df[area_column].apply(prepare_area_column)
    main_df = main_df[main_df["area_code"].apply(lambda x: x.isdigit())]
    main_df.to_csv(f"{DATA_DIRECTORY_PATH}/pre_derived_lpa/yearly/{year}.csv", index=False)

In [15]:
final_df = pd.DataFrame()

for year, data_path in lpa_data_paths.items():
    main_df = pd.read_csv(data_path)
    main_df = main_df.assign(year=year, count=1)
    main_df["area_code"] = main_df[addr_column].apply(lambda x: get_city_code(parser, x))
    main_df["area"] = main_df["area_code"].apply(get_area_from_area_code)
    main_df["unit_price"] = main_df[price_per_unit_area_column].apply(prepare_area_column)
    main_df["traded_area"] = main_df[area_column].apply(prepare_area_column)
    main_df = main_df[main_df["area_code"].apply(lambda x: x.isdigit())]
    
    df = main_df[required_columns]
    df = (
        df.groupby(["year", "area_code", "area"])
        .apply(custom_aggregate)
        .reset_index()
    )
    df = df.sort_values(by=["year", "area_code"], ascending=[False, True]).reset_index(drop=True)
    df.to_csv(f"{DATA_DIRECTORY_PATH}/derived_lpa/yearly/{year}.csv", index=False)
    final_df = pd.concat([final_df, df])
    

KeyError: "['price_per_unit_area', 'area'] not in index"

In [10]:
final_df.to_csv(f"{DATA_DIRECTORY_PATH}/derived_lpa/wappa.csv", index=False)
